In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

# import lightgbm as lgb
# from sklearn.model_selection import *
# from sklearn.metrics import mean_squared_error, make_scorer
# from scipy.stats import mode, skew, kurtosis, entropy
# from sklearn.ensemble import ExtraTreesRegressor

# import matplotlib.pyplot as plt
# import seaborn as sns

# import dask.dataframe as dd
# from dask.multiprocessing import get

# from tqdm import tqdm, tqdm_notebook
# tqdm.pandas(tqdm_notebook)

import gc
gc.enable()

import time

/home/ec2-user/miniconda3/envs/datascience/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ec2-user/miniconda3/envs/datascience/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
def load_data(conf):
    st = time.time()
    if os.path.exists(f'{conf}.feather'):
        out = pd.read_feather(f'{conf}.feather')
        print(f'{conf} data loaded from feather {time.time()-st:.2f} s!')
    else:
        out = pd.read_csv(f'./{conf}.csv.zip')
        print(f'{conf} data loaded from csv {time.time()-st:.2f} s!')
        out.to_feather(f'{conf}.feather')
        print(f'{conf} data save to feather {time.time()-st:.2f} s!')  
    return out 
train = load_data('train')
test = load_data('test')

/home/ec2-user/miniconda3/envs/datascience/lib/python3.6/site-packages/pandas/io/feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)


train data loaded from feather 0.65 s!
test data loaded from feather 8.15 s!


In [3]:
transact_cols = [f for f in train.columns if f not in ["ID", "target"]]
y = np.log1p(train["target"]).values

In [4]:
cols = ['f190486d6', '58e2e02e6', 'eeb9cd3aa', '9fd594eec', '6eef030c1',
       '15ace8c9f', 'fb0f5dbfe', '58e056e12', '20aa07010', '024c577b9',
       'd6bb78916', 'b43a7cfd5', '58232a6fb', '1702b5bf0', '324921c7b', 
       '62e59a501', '2ec5b290f', '241f0f867', 'fb49e4212',  '66ace2992',
       'f74e8f13d', '5c6487af1', '963a49cdc', '26fc93eb7', '1931ccfdd', 
       '703885424', '70feb1494', '491b9ee45', '23310aa6f', 'e176a204a',
       '6619d81fc', '1db387535', 'fc99f9426', '91f701ba2',  '0572565c2',
       '190db8488',  'adb64ff71', 'c47340d97', 'c5a231d81', '0ff32eb98'] 

In [5]:
%%time
test["target"] = train["target"].mean()

all_df = pd.concat([train[["ID", "target"] + cols], test[["ID", "target"]+ cols]]).reset_index(drop=True)
all_df.head()
all_df.shape

CPU times: user 837 ms, sys: 1.76 s, total: 2.6 s
Wall time: 1.85 s


In [7]:
from multiprocessing import Pool, cpu_count

def _time_series_info(row):
    try:
        id_1st_nz = row.nonzero()[0][0]
        value_1st_nz = str(row[id_1st_nz])
    except:
        return '0_0.0'   
    return str(id_1st_nz)+'_'+value_1st_nz

def _join2str(df):
    return df.apply(lambda x: "_".join(x.round(2).astype(str)), axis=1)

def _get_leak(df, cols, lag=0, n_thread=4):
    """
    Get leaked data
    """
    st = time.time()
    
    df_split = np.array_split(df[cols[lag+2:]], n_thread)
    df_shift_split = np.array_split(df[cols].shift(lag+2, axis=1)[cols[lag+2:]], n_thread)
    
    print(f'Shift columns: {time.time()-st:.2f} seconds!')
    with Pool(processes=n_thread) as p:
        result1 = p.map(_join2str, df_split)
        result2 = p.map(_join2str, df_shift_split)
        
    series_str = pd.concat(list(result1), ignore_index=True)
    series_shifted_str = pd.concat(list(result2), ignore_index=True)
    print(f'Create time series strings before and after shift: {time.time()-st:.2f} seconds!')
    
    st = time.time()
    series_dict = {}
    for i in range(len(series_str)):
        key = series_str[i]
        if key in series_dict.keys():
            continue
        series_dict[key] = i
    print(f'Create dictionary for faster search: {time.time()-st:.2f} seconds!')
    
    st = time.time()
    target_vals = series_shifted_str.apply(lambda x: df.loc[series_dict[x], cols[lag]] 
                                                   if x in series_dict else 0)
    print(f'Matching process finished: {time.time()-st:.2f} seconds!')
    return target_vals

def get_all_leak(df, cols=None, nlags=15):
    """
    We just recursively fetch target value for different lags
    """
    df =  df.copy()
    
    for i in range(nlags):
        print("Processing lag {}".format(i))
        df["leaked_target_"+str(i)] = _get_leak(df, cols, i)
    return df

In [8]:
%%time
NLAGS = 38 #Increasing this might help push score a bit
leaky_cols = ["leaked_target_"+str(i) for i in range(NLAGS)]
all_df = get_all_leak(all_df, cols=cols, nlags=NLAGS)

Processing lag 0
Shift columns: 0.05 seconds!
Create time series strings before and after shift: 5.88 seconds!
Create dictionary for faster search: 0.43 seconds!
Matching process finished: 0.22 seconds!
Processing lag 1
Shift columns: 0.06 seconds!
Create time series strings before and after shift: 5.79 seconds!
Create dictionary for faster search: 0.43 seconds!
Matching process finished: 0.21 seconds!
Processing lag 2
Shift columns: 0.07 seconds!
Create time series strings before and after shift: 5.70 seconds!
Create dictionary for faster search: 0.42 seconds!
Matching process finished: 0.21 seconds!
Processing lag 3
Shift columns: 0.06 seconds!
Create time series strings before and after shift: 5.79 seconds!
Create dictionary for faster search: 0.42 seconds!
Matching process finished: 0.22 seconds!
Processing lag 4
Shift columns: 0.06 seconds!
Create time series strings before and after shift: 5.69 seconds!
Create dictionary for faster search: 0.42 seconds!
Matching process finished:

In [9]:
%%time
## post-processing
def _get_likely(row):
    tar = row.target
    nz_idx = row[leaky_cols].nonzero()[0]
    try:
        nz_cnt_df = row[leaky_cols][nz_idx].value_counts().reset_index()
        col_name = nz_cnt_df.columns[1]
        sorted_df = nz_cnt_df.sort_values(by=col_name, ascending=False)
        if sorted_df.iloc[0, 1] == 1:
            likely = 0.0
            prob = 0.0
            cnt = 0
        else:
            likely = sorted_df.iloc[0, 0]
            cnt = sorted_df.iloc[0, 1]
            prob = sorted_df.iloc[0, 1]/sorted_df[col_name].sum()
            
    except:
        likely = 0.0
        prob = 0.0
        cnt = 0
    return likely, cnt, prob

likely_args = all_df.apply(lambda x: _get_likely(x), axis=1)    

CPU times: user 2min 45s, sys: 570 ms, total: 2min 45s
Wall time: 2min 45s


In [10]:
## Save the most likely values to feather
likely_df = pd.DataFrame()
likely_df['ID'] = all_df['ID']
likely_df['likely_value'] = likely_args.apply(lambda x:x[0])
likely_df['likely_cnt'] = likely_args.apply(lambda x:x[1])
likely_df['likely_prob'] = likely_args.apply(lambda x:x[2])
likely_df.to_feather('most_likely_value.feather')
gc.collect()

3542

In [11]:
%%time
all_df['likely_value'] = likely_args.apply(lambda x:x[0])

train_processed = train.join(all_df.set_index("ID")[leaky_cols+['likely_value']], on="ID", how="left")
test_processed = test.join(all_df.set_index("ID")[leaky_cols+['likely_value']], on="ID", how="left")

train_processed["nonzero_mean"] = train[transact_cols].apply(lambda x: np.expm1(np.log1p(x[x!=0]).mean()), axis=1)
test_processed["nonzero_mean"] = test[transact_cols].apply(lambda x: np.expm1(np.log1p(x[x!=0]).mean()), axis=1)

gc.collect()

CPU times: user 27.5 s, sys: 3.07 s, total: 30.6 s
Wall time: 29.6 s


In [13]:
%%time
#We start with 1st lag target and recusrsively fill zero's
def _get_prediction(row):
    if row.likely_value != 0:
        return row.likely_value
    else:
        return row.nonzero_mean
    
train_processed['predict'] = train_processed.apply(lambda x: _get_prediction(x), axis=1)
test_processed['predict'] = test_processed.apply(lambda x: _get_prediction(x), axis=1)

CPU times: user 53.2 s, sys: 8.08 s, total: 1min 1s
Wall time: 1min


In [15]:
sub = test[["ID"]]
sub["target"] = test_processed["predict"]

if not os.path.exists('submissions'):
    os.mkdir('submissions')
    
sub.to_csv('submissions/baseline_submission_with_leaks_'+'_'.join(time.ctime().split())+'.csv', index=False)

/home/ec2-user/miniconda3/envs/datascience/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
